## Rozwiązywanie układów równań liniowych 

### Zadanie 1 
Zaimplemenuj metodę eliminacji Gaussa bez pivotingu i z pivotingiem dla układu równań o dowolnym rozmiarze.  

In [14]:
import numpy as np
from tabulate import tabulate


# ============================================================
# Gaussian elimination without pivoting
# ============================================================

def forward_elimination(A, b):
    n = len(A)
    for k in range(0, n-1):
        for i in range(k+1, n):
            factor = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] = A[i][j] - factor * A[k][j]
            b[i] = b[i] - factor * b[k]
    return A, b


def back_substitution(A, b):
    n = len(A)
    x = [0 for _ in range(n)]
    x[n-1] = b[n-1] / A[n-1][n-1]
    for k in range(n-2, -1, -1):
        sums = b[k]
        for j in range(k+1, n):
            sums = sums - A[k][j] * x[j]
        x[k] = sums / A[k][k]
    return x


def gauss_without_pivot(A, b):
    for i in range(len(A)):
        if len(A) != len(A[i]):
            raise ZeroDivisionError('Division by zero will occur; pivoting currently not supported')
    A, b = forward_elimination(A, b)
    return back_substitution(A, b)


# ============================================================
# Gaussian elimination with pivoting
# ============================================================

def gauss_with_pivot(A, b):
    n = len(A)
    M = A
    i = 0

    for x in A:
        x.append(b[i])
        i += 1

    for k in range(n):
        for i in range(k, n):
            if abs(M[i][k]) > abs(M[k][k]):
                M[k], M[i] = M[i], M[k]

        for j in range(k+1, n):
            q = M[j][k] / M[k][k]
            for m in range(k, n+1):
                M[j][m] -= q * M[k][m]

    x = [0 for _ in range(n)]

    x[n-1] = M[n-1][n] / M[n-1][n-1]
    for i in range(n-1, -1, -1):
        z = 0
        for j in range(i+1, n):
            z += M[i][j]*x[j]
        x[i] = (M[i][n] - z) / M[i][i]
    return x

### Zadanie 2
Zademonstruj działanie algorytmu na macierzy o rozmiarze 5 x 5. Zademonstruj w jakiej sytuacji potrzebny jest pivoting i jak działa.

In [17]:
A = \
    [[0.00000000000000012, 4200, 34, 65, 9012],
     [0.5, 0.00000056, 2320, 53, 456],
     [1, 17.000005, 7, 0.000000000004, 458],
     [38, 9, 896, 0.000000006, 58],
     [586, 6, 3857, 56, 0.000065]]
b = \
    [7, 3.901, 6, 6, 858]
x1 = gauss_without_pivot(A, b)

C = \
    [[0.00000000000000012, 4200, 34, 65, 9012],
     [0.5, 0.00000056, 2320, 53, 456],
     [1, 17.000005, 7, 0.000000000004, 458],
     [38, 9, 896, 0.000000006, 58],
     [586, 6, 3857, 56, 0.000065]]
d = \
    [7, 3.901, 6, 6, 8585]
x2 = gauss_with_pivot(C, d)

E = \
    [[0.00000000000000012, 4200, 34, 65, 9012],
     [0.5, 0.00000056, 2320, 53, 456],
     [1, 17.000005, 7, 0.000000000004, 458],
     [38, 9, 896, 0.000000006, 58],
     [586, 6, 3857, 56, 0.000065]]
f = \
    [7, 3.901, 6, 6, 8585]
x3 = np.linalg.solve(E, f)

tab = [["Without pivoting"] + x1,
       ["With pivoting"] + x2,
       ["numpy.linalg"] + list(x3)]

print(tabulate(tab, headers=["Type", "x1", "x2", "x3", "x4", "x5"]))

Type                    x1           x2          x3         x4           x5
----------------  --------  -----------  ----------  ---------  -----------
Without pivoting  -44.4089  -0.00113169  -0.0105263   0.381579  -0.00140831
With pivoting      16.2984  -0.462624    -0.680213   29.6515     0.00508228
numpy.linalg       16.2984  -0.462624    -0.680213   29.6515     0.00508228


Powodem błędu w zwykłym podejściu (bez pivotingu) jest duża różnica rzędów wielkości między wartościami (współczynnikami) macierzy. Poszczególne wiersze dzielimy przez bardzo mała liczbę (czyli mnożymy przez dużą), więc błędy zaokrągleń stają sie duże w stosunku do współczynników oryginalnej macierzy. Abo uniknąć tego problemu używamy pivota, który równoważy ten efekt poprzez wybieranie za każdym razem jako pivot liczbę o największej wartości bezwzględnej w danej kolumnie.

### Zadanie 3
Podaj teorytyczną złożoność obliczeniową algorytmu eliminacji Gaussa. Przeprowadź testy wydajności swojego algorytmu sprawdzając jego działanie dla różnych rozmiarów macierzy (testy powinny być wykonane poza środowiskiem jupyter). Aby wygenerować układ równań, wygeneruj wektor rozwiązań i macierz współczynników losując wartości (skorzystaj z funkcji poznanych w Ćwiczeniu 2) i następnie oblicz wektor wyrazów wolnych. 

Teoretyczna złożoność algorytmu eliminacji Gaussa wynosi O($n^3$).
Poniżej zamieszczam kod użyty do wykonania pomiarów. Wykres znajduje się w osobnym pliku PDF.

In [ ]:
import matplotlib.pyplot as plt
import time


def measure():
    tab1 = []
    tab2 = []
    x_axis = []
    for n in range(100, 400):
        A = np.random.randint(-10000, 10000, size=(n, n))
        b = np.random.rand(len(A))
        C = A.copy()
        d = b.copy()
        try:
            start_time_1 = time.time()
            gauss_with_pivot(A.tolist(), b.tolist())
            stop_time_1 = time.time()
            start_time_2 = time.time()
            gauss_without_pivot(C.tolist(), d.tolist())
            stop_time_2 = time.time()
            t1 = stop_time_1 - start_time_1
            t2 = stop_time_2 - start_time_2
            tab1.append(t1)
            tab2.append(t2)
            x_axis.append(n)
            print(f"{n}x{n} With pivot: {t1}")
            print(f"{n}x{n} Without pivot: {t2}\n")
        except ZeroDivisionError:
            pass
    return tab1, tab2, x_axis


# tab1, tab2, x_axis = measure()

plt.plot(x_axis, tab1, 'b.', label="With pivoting")
plt.plot(x_axis, tab2, 'r.', label="Without pivoting")
plt.xlabel("Square matrix size")
plt.ylabel("Time [s]")
plt.title("Time of evaluation fro Gaussian elimination")
plt.legend()
plt.grid()
plt.show()